# Introduction to Cross Validation

In this lecture series we will do a much deeper dive into various methods of cross-validation. As well as a discussion on the general philosphy behind cross validation. A nice official documentation guide can be found here: https://scikit-learn.org/stable/modules/cross_validation.html

In [1]:
# import library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scienceplots

In [2]:
df = pd.read_csv('..\data\Advertising.csv')

In [3]:
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [4]:
# create X and y
X = df.drop('sales', axis=1)
y = df['sales']

# Train | test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size= 0.3, random_state = 101)

# scale data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

**create ridge(l2)**

In [5]:
from sklearn.linear_model import Ridge

In [6]:
ridge_model = Ridge(alpha =100)

In [7]:
ridge_model.fit(X_train, y_train)

Ridge(alpha=100)

In [8]:
y_pred = ridge_model.predict(X_test)

**Evaluation**

In [9]:
from sklearn.metrics import mean_squared_error

In [10]:
mean_squared_error(y_test, y_pred)

7.341775789034128

**Adjust parameters and Re-Evaluate**

In [11]:
model = Ridge(alpha=1)

In [12]:
model.fit(X_train, y_train)

Ridge(alpha=1)

In [13]:
y_pred = model.predict(X_test)

**Another evaluation**

In [14]:
mean_squared_error(y_test, y_pred)

2.3190215794287523

Much better! We could repeat this until satisfied with performance metrics. (We previously showed RidgeCV can do this for us, but the purpose of this lecture is to generalize the CV process for any model).

----
----
----

----
----
----
## Train | Validation | Test Split Procedure 

This is often also called a "hold-out" set, since you should not adjust parameters based on the final test set, but instead use it *only* for reporting final expected performance.

0. Clean and adjust data as necessary for X and y
1. Split Data in Train/Validation/Test for both X and y
2. Fit/Train Scaler on Training X Data
3. Scale X Eval Data
4. Create Model
5. Fit/Train Model on X Train Data
6. Evaluate Model on X Evaluation Data (by creating predictions and comparing to Y_eval)
7. Adjust Parameters as Necessary and repeat steps 5 and 6
8. Get final metrics on Test set (not allowed to go back and adjust after this!)

In [15]:
X = df.drop('sales', axis=1)
y = df['sales']

In [16]:

from sklearn.model_selection import train_test_split

X_train, X_OTHER, y_train, y_OTHER = train_test_split(X,y, test_size =0.3, random_state = 101)


# Remaining 30% is split into evaluation and test sets
# Each is 15% of the original data size

X_eval, X_test, y_eval, y_test = train_test_split(X_OTHER, y_OTHER, test_size= 0.5, random_state = 101)

In [17]:
# Scale Data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_eval = scaler.transform(X_eval)
X_test = scaler.transform(X_test)

**Create Model**

In [18]:
from sklearn.linear_model import Ridge

In [19]:
model = Ridge(alpha =100)

In [20]:
model.fit(X_train, y_train)

Ridge(alpha=100)

In [21]:
y_eval_pred = model.predict(X_eval)

**Evaluation**

In [22]:
from sklearn.metrics import mean_squared_error

In [23]:
mean_squared_error(y_eval, y_eval_pred)

7.320101458823869

**Adjust Paramters and Re-Evaluate**

In [24]:
model = Ridge(alpha= 1)

In [25]:
model.fit(X_train, y_train)

Ridge(alpha=1)

In [26]:
y_eval_pred = model.predict(X_eval)

**Another Evaluation**

In [27]:
mean_squared_error(y_eval, y_eval_pred)

2.383783075056987

**Final Evaluation(can no longer edit parameter after this)**

In [28]:
y_final_test_pred =model.predict(X_test)

In [29]:
mean_squared_error(y_test, y_final_test_pred)

2.2542600838005176

----
----
----
## Cross Validation with cross_val_score

----

<img src="grid_search_cross_validation.png">

----

In [30]:
# create X and y
X = df.drop('sales', axis =1)
y = df['sales']

# Train | Test Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.3, random_state =101)

# Scale Data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [31]:
model = Ridge(alpha = 100)

In [32]:
from sklearn.model_selection import cross_val_score

In [33]:
# scoring options:

scores = cross_val_score(model, X_train, y_train, scoring='neg_mean_squared_error', cv=5)

In [34]:
scores

array([ -9.32552967,  -4.9449624 , -11.39665242,  -7.0242106 ,
        -8.38562723])

In [35]:
# Average of the MSE scores(we set back positive)
abs(scores.mean())

8.215396464543607

**Adjust model based on matrics**

In [36]:
model = Ridge(alpha =1)

In [37]:
scores = cross_val_score(model, X_train, y_train, scoring='neg_mean_squared_error', cv=5)

In [38]:
# Average of the MSE scores(we set back positive)
abs(scores.mean())

3.344839296530696

**Final Evaluation(can no longer edit parameter after this)**

In [39]:
model.fit(X_train, y_train)

Ridge(alpha=1)

In [40]:
y_final_model_pred = model.predict(X_test)

In [41]:
mean_squared_error(y_test, y_final_model_pred)

2.3190215794287523

----
----
----

# Cross Validation with cross_validate

The cross_validate function differs from cross_val_score in two ways:

It allows specifying multiple metrics for evaluation.

It returns a dict containing fit-times, score-times (and optionally training scores as well as fitted estimators) in addition to the test score.

For single metric evaluation, where the scoring parameter is a string, callable or None, the keys will be:
        
        - ['test_score', 'fit_time', 'score_time']

And for multiple metric evaluation, the return value is a dict with the following keys:

    ['test_<scorer1_name>', 'test_<scorer2_name>', 'test_<scorer...>', 'fit_time', 'score_time']

return_train_score is set to False by default to save computation time. To evaluate the scores on the training set as well you need to be set to True.

In [42]:
# create X and y
X = df.drop('sales', axis=1)
y = df['sales']

# Train Test Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 101)

# Scale Data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [43]:
model = Ridge(alpha=100)

In [44]:
from sklearn.model_selection import cross_validate

In [45]:
scores = cross_validate(model, X_train, y_train, 
                        scoring = ['neg_mean_squared_error', 'neg_mean_absolute_error', 'max_error'], cv=5 )

In [46]:
scores

{'fit_time': array([0.0030005 , 0.00100088, 0.00100279, 0.00220919, 0.00099874]),
 'score_time': array([0.00299716, 0.00099921, 0.00199986, 0.0010016 , 0.00200009]),
 'test_neg_mean_squared_error': array([ -9.32552967,  -4.9449624 , -11.39665242,  -7.0242106 ,
         -8.38562723]),
 'test_neg_mean_absolute_error': array([-2.31243044, -1.74653361, -2.56211701, -2.01873159, -2.27951906]),
 'test_max_error': array([ -6.44988486,  -5.58926073, -10.33914027,  -6.61950405,
         -7.75578515])}

In [47]:
pd.DataFrame(scores)

,fit_time,score_time,test_neg_mean_squared_error,test_neg_mean_absolute_error,test_max_error
0,0.003000,0.002997,-9.325530,-2.312430,-6.449885
1,0.001001,0.000999,-4.944962,-1.746534,-5.589261
2,0.001003,0.002000,-11.396652,-2.562117,-10.339140
3,0.002209,0.001002,-7.024211,-2.018732,-6.619504
4,0.000999,0.002000,-8.385627,-2.279519,-7.755785


In [48]:
pd.DataFrame(scores).mean()

fit_time                        0.001642
score_time                      0.001800
test_neg_mean_squared_error    -8.215396
test_neg_mean_absolute_error   -2.183866
test_max_error                 -7.350715
dtype: float64

**Adjust model based on matrics**

In [49]:
model = Ridge(alpha =1)

In [50]:
scores = cross_validate(model, X_train, y_train,
                        scoring =['neg_mean_squared_error','neg_mean_absolute_error', 'max_error'], cv=5)

In [51]:
pd.DataFrame(scores).mean()

fit_time                        0.001800
score_time                      0.001637
test_neg_mean_squared_error    -3.344839
test_neg_mean_absolute_error   -1.319685
test_max_error                 -5.161145
dtype: float64

**Final Evaluation (Can no longer edit parameters after this!)**

In [52]:
model.fit(X_train, y_train)

Ridge(alpha=1)

In [53]:
y_final_eval_pred = model.predict(X_test)

In [54]:
mean_squared_error(y_test, y_final_eval_pred)

2.3190215794287523